<a href="https://colab.research.google.com/github/Khuzamaalk/T5_BootCamp/blob/main/M_Text_Summarization_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning a Model for Summarization Task

In this task, you will load, preprocess, and fine-tune a T5 model on a dataset of news articles for a summarization task. Follow the steps below carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `UBC-NLP/AraT5-base` if you face any problem you can use `google-t5/t5-small` but the first one is the correct one for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/news_articles_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

from datasets import load_dataset

dataset = load_dataset('CUTD/news_articles_df')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


news_articles_df.csv:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8378 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'summarizer', 'text'],
        num_rows: 8378
    })
})

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(dataset['train'])
df.head()

,Unnamed: 0,summarizer,text
0,0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...
1,1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...",تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...
2,2,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...,احتضن جناح تونس القريه الدوليه للافلام بمدينه ...
3,3,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...,شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...
4,4,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...,نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...


In [ ]:
print(f'first text:\n {df["text"]}')
print("\n----------------------------------------\n")
print(f'first summary:\n {df["summarizer"]}')

first text:
 0       اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...
1       تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...
2       احتضن جناح تونس القريه الدوليه للافلام بمدينه ...
3       شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...
4       نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...
                              ...                        
8373    تاجل الاضراب العام قطاع الصحه مقررا تنفيذه الي...
8374    كشف الناشطان كريم نوار وعفيف زقيه اشرفا عمليه ...
8375    فرقه الابحاث والتفتيش للحرس الوطني بطبلبه ولاي...
8376    قرر الاهالي بمناطق هيشر وعين القارصي والغولايث...
8377    تمكنت وحدات الحرس الوطني بمحطه الاستخلاص ببرج ...
Name: text, Length: 8378, dtype: object

----------------------------------------

first summary:
 0       \nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...
1       \nتحصل كتاب "المصحف وقراءاته" الذي ألفه باحثون...
2       تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...
3       واستأجرت صاحبة المشروع المحامية والكاتبة سيران...
4       \nنعت وزار

In [ ]:
df = df.drop(columns= 'Unnamed: 0')
df.head()

,summarizer,text
0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...
1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...",تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...
2,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...,احتضن جناح تونس القريه الدوليه للافلام بمدينه ...
3,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...,شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...
4,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...,نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...


## Step 2: Load the Pretrained Tokenizer

Initialize a tokenizer from the gevin model checkpoint.

In [ ]:
from transformers import AutoTokenizer

checkpoint = "UBC-NLP/AraT5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Step 3: Preprocess the Dataset

Define a preprocessing function that adds a prefix ("summarize:") to each input if needed and tokenizes the text for the model. The labels will be the tokenized summaries.

In [ ]:
prefix = "summarize: "

def preprocess(texts):

  inputs = [prefix + doc for doc in texts['text']]

  model_inputs = tokenizer(inputs, max_length= 512, truncation= True)
  labels = tokenizer(text_target= texts['summarizer'], max_length= 512, truncation= True)

  model_inputs['labels'] = labels['input_ids']

  return model_inputs

In [ ]:
tokenized_data = dataset.map(preprocess, batched = True)

Map:   0%|          | 0/8378 [00:00<?, ? examples/s]

## Step 4: Define the Data Collator

Use a data collator designed for sequence-to-sequence models, which dynamically pads inputs and labels.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

## Step 5: Load the Pretrained Model

Load the model for sequence-to-sequence tasks (summarization).

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

tf_model.h5:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at UBC-NLP/AraT5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


## Step 6: Define Training Arguments

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
train_size = int(0.7 * len(tokenized_data["train"]))
validation_size = int(0.15 * len(tokenized_data["train"]))
test_size = int(0.15 * len(tokenized_data["train"]))

#update the tokenized data with the new splits
tokenized_data = {"train": tokenized_data["train"].select(range(train_size)),
                     "validation": tokenized_data["train"].select(range(train_size, train_size + validation_size)),
                     "test": tokenized_data["train"].select(range(train_size + validation_size, train_size + validation_size + test_size))}

In [ ]:
batch_size=4
tf_train_set = model.prepare_tf_dataset(
    tokenized_data["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_data["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_data["test"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [ ]:
model.compile(optimizer=optimizer)

## Step 7: Initialize the Trainer

Use the `Seq2SeqTrainer` class to train the model.

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    num_train_epochs=3,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tf_train_set,
    eval_dataset=tf_test_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


AttributeError: 'TFT5ForConditionalGeneration' object has no attribute 'to'

## Step 8: Fine-tune the Model

Train the model using the specified arguments and dataset.

In [ ]:
trainer.train()

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=1)

1466/1466 [==============================] - 787s 537ms/step - loss: 15.4168 - val_loss: 11.2171


## Step 9: Inference

Once the model is trained, perform inference on a sample text to generate a summary. Use the tokenizer to process the text, and then feed it into the model to get the generated summary.

In [ ]:
model.save_pretrained("my_model")

In [ ]:
tokenizer.save_pretrained("/content/my_model")

('/content/my_model/tokenizer_config.json',
 '/content/my_model/special_tokens_map.json',
 '/content/my_model/spiece.model',
 '/content/my_model/added_tokens.json',
 '/content/my_model/tokenizer.json')

In [ ]:
text = '''أتشعرُ أنّك مرهقٌ جداً يا فتى؟ متعبٌ من كلّ شيءٍ، وساخطٌ على كلّ شيءْ، تبدُو لِي كذلك، وعيناكَ الضيّقتانِ، تزيدانِ من حدّتكْ، كلّما اكتملتْ تلكَ العقدةُ الّتي تعلُو وجهكْ.

اهدأ، فأنا أستطيعُ أنْ أتفهّم غضبكْ ونقمتكَ على الحياةِ كلّها، وأنتَ تجلسُ كلّ صّباحٍ في هذهِ الزاويةِ المعتمةِ منْ هذا الكوكبِ المقفرِ، تنتظرُ منْ يمرُّ من هُنا راغباً في مسحِ حذائهِ.

تشعرُ بالخجلْ أليسَ كذلك؟ أو ربّما تشعرُ أنّك مطحونٌ في ركنٍ منسيٍ من هذا الكونْ، تشعرُ بالرّغبةِ في البكاءْ، كلّما ناولكَ أحدهمْ نظيرَ عملكْ، أنا أفهمكْ حقاً، لكنّي أفهمُ أيضاً أنّنا لا نختارُ ما نحنُ عليهْ، بينما نستطيعُ تغييرهُ بأيدينا مسْتقبلاً، أنتَ تبْنِي نفسكْ، فلا تسْتهنْ بكلّ الّذي تقُومْ به الآن.

غداً حينَ ستكبرْ، ستدركُ أنّك قدْ صقلتَ الرّجولة فيكَ مبكراً جداً، وأنّ الطّفولةَ الّتي حُرمتَ جنّتها، ستعوّضُ برجولةٍ مكتملةٍ وقادرةٍ على مواجهةِ صُعوباتِ الحياة، أنتَ تصنعُ من نفسكَ الآن رجلاً، وقليلُون جداً همُ الرجالُ على هذا الكوكبْ.

يا صّغيرِي، لا تخجلْ منْ نفسكْ أبداً، فأنتَ الآن درسٌ للعالمِ كلّه.'''

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="/content/my_model")
summarizer(text)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/my_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'summary_text': 'طعنة طعنة، Palaistemptemptemptemptemp ميناء ميناءtemptemptemptemptemp ميناء،،'}]